In [1]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import *

connection_parameters = {
    "account": "DRNHDAY-JL77246",
    "user": "NASRULHASAN",
    "password" : "***REMOVED***",
    "role" : "ACCOUNTADMIN",
    "warehouse" : "SNOWFLAKE_LEARNING_WH",
    "database" : "SNOWFLAKE_LEARNING_DB",
    "schema" : "NASRULHASAN_LOAD_SAMPLE_DATA_FROM_S3"
}

session = Session.builder.configs(connection_parameters).create()

In [4]:
# Create file format
session.sql("""
CREATE OR REPLACE FILE FORMAT my_csv_format
  TYPE = 'CSV'
  FIELD_DELIMITER = ','
  SKIP_HEADER = 1
  NULL_IF = ('NULL', 'null')
""").collect()

'File format MY_CSV_FORMAT successfully created.'

In [5]:
# create a stage from
# s3://snowflake-workshop-lab/citibike-trips-csv/trips_2020_07.csv
session.sql("""
CREATE OR REPLACE STAGE my_s3_stage
  URL = 's3://snowflake-workshop-lab/citibike-trips-csv/'
  FILE_FORMAT = my_csv_format
""").collect()

print("✅ Stage created")


✅ Stage created


In [6]:
session.sql("""
CREATE OR REPLACE TABLE citibike_trips (
    tripduration INT,
    starttime TIMESTAMP,
    stoptime TIMESTAMP,
    start_station_name STRING,
    end_station_name STRING,
    usertype STRING
)
""").collect()

print("✅ Table created")

✅ Table created


In [7]:
session.sql("""
COPY INTO citibike_trips
FROM @my_s3_stage/trips_2020_07.csv
FILE_FORMAT = (FORMAT_NAME = my_csv_format)
ON_ERROR = 'CONTINUE'
""").collect()

print("✅ Data loaded from S3 into Snowflake")

✅ Data loaded from S3 into Snowflake


In [8]:
df = session.table("citibike_trips")
df.show(5)


------------------------------------------------------------------------------------------------------
|"TRIPDURATION"  |"STARTTIME"  |"STOPTIME"  |"START_STATION_NAME"  |"END_STATION_NAME"  |"USERTYPE"  |
------------------------------------------------------------------------------------------------------
|                |             |            |                      |                    |            |
------------------------------------------------------------------------------------------------------

